# OpenAI Function Calling


**Notes**:
- LLM은 항상 동일한 결과를 반환하지 않습니다. 여러분이 노트북에서 확인할 결과는 영상에서의 결과와 다를 수 있습니다.
- 노트북의 결과는 영구적으로 저장되는 것이 아닙니다. 결과를 저장하고 싶다면 노트북을 로컬 기기에 다운로드 받으세요.

In [ ]:
import os
import openai

# 로컬에서 직접 실행하는 경우, openai api key를 직접 입력해야 합니다.
# 저는 .txt 파일에 key를 저장하여 사용하고 있습니다.
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
import json

# 같은 날씨를 반환하도록 하드 코딩된 더미 함수 예제입니다.
# 실제로는 백엔드 API 또는 외부 API가 될 수 있습니다.
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit, # 단위 정보
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
# 함수 정의. json 형태로 정의합니다.
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location", # 함수에 대한 설명
        "parameters": { # 함수에 포함되는 변수들
            "type": "object",
            "properties": {
                "location": { # 변수1: 위치
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA", # 변수1에 대한 설명
                },
                "unit": { # 변수2: 단위
                    "type": "string", 
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["location"], # 필수 변수
        },
    }
]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [ ]:
import openai

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [ ]:
print(response)

In [ ]:
# response 출력 결과
{
  "id": "chatcmpl-8Fc9tztf0SPaO04MAHUwgbqos9Gh4",
  "object": "chat.completion",
  "created": 1698732293,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 18,
    "total_tokens": 100
  }
}

In [ ]:
# 여기서 추출하는 message는 함수로 전달될 것입니다.
response_message = response["choices"][0]["message"]

In [ ]:
response_message

In [ ]:
# null
response_message["content"]

In [ ]:
# 함수 호출 정보
response_message["function_call"]

In [ ]:
# 함수에 전달할 arguments를 json 형식으로 불러오기
json.loads(response_message["function_call"]["arguments"])

In [ ]:
args = json.loads(response_message["function_call"]["arguments"])

In [ ]:
# 함수 호출하며 arguments 전달하기
get_current_weather(args)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [ ]:
# 우리가 정의한 함수에는 'hi'에 답변하는 내용이 포함되어 있지 않습니다.
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

In [ ]:
print(response)

In [ ]:
# `function_call`을 `auto`로 설정하면, 함수를 자동으로 호출합니다.
# 위에서와 마찬가지로 호출할만한 함수가 없어서 모델 스스로 답변을 생성해낸 것을 확인할 수 있습니다.
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

In [ ]:
# `function_call`을 `none`으로 설정하면, 함수를 호출하지 않고 모델이 직접 답변을 생성해냅니다.
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

In [ ]:
# `function_call`이 `none`이면, 함수를 호출하지 않고 모델이 직접 답변을 생성해냅니다.
# 모델은 Boston의 현재 날씨에 대해 알지 못하므로 답변할 수 없다고 답변합니다.
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

In [ ]:
# 아래와 같은 형태로 특정 함수를 지정하여 호출할 수도 있습니다.
# 그러나 날씨와 관련되지 않은 내용을 입력했기 때문에, 예시로 입력했던 San Francisco의 날씨를 반환합니다.
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

In [ ]:
messages.append(response["choices"][0]["message"])

In [ ]:
args = json.loads(response["choices"][0]["message"]['function_call']['arguments'])
observation = get_current_weather(args)

In [ ]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response)